In [ ]:
# This is based on Fujisan's kernel on Invasive Species Monitoring

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import glob
from glob import glob

# Any results you write to the current directory are saved as output.

In [ ]:
train_path = '../input/train'
path_name = train_path + '/**/*.jpg'

In [ ]:
train_image_paths = glob(path_name, recursive=True)

In [ ]:
train_image_paths[:10]

In [ ]:
train_categories = list(map(os.path.basename,train_image_paths))

In [ ]:
train_categories[:3]

In [ ]:
labels =[]
for category in train_categories:
    labels.append(category[:3])

In [ ]:
labels[:10]

In [ ]:
len(labels)

In [ ]:
len(train_image_paths)

In [ ]:
num_classes = len(np.unique(labels))
num_classes

# Labels Data Preparation

## Encoding labels

In [ ]:
#Encode labels with value between 0 and n_classes-1.
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
loadedLabels = np.asarray(labels)
encoder.fit(loadedLabels)
encoded_loadedLabels = encoder.transform(loadedLabels)

## Convert to One Hot Encoding Labels

In [ ]:
# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
from keras.utils.np_utils import to_categorical
labels_Hot = to_categorical(encoded_loadedLabels, num_classes = num_classes)

In [ ]:
labels_Hot[:3]

In [ ]:
labels[:3]

# Create the Dataframe for Datagenerators

In [ ]:
df= pd.DataFrame()

In [ ]:
df

In [ ]:
df['path']=train_image_paths

In [ ]:
df['path'].head()

In [ ]:
df['labels'] = list(labels_Hot)

In [ ]:
df.head()

# Create Data Generators

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
IMG_SIZE = (128, 128)
core_idg = ImageDataGenerator()

In [ ]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'sparse',
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen

In [ ]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, 
                                   test_size = 0.25, 
                                   random_state = 2018)

In [ ]:
len(train_df)

In [ ]:
len(valid_df)

In [ ]:
train_gen = flow_from_dataframe(core_idg, train_df, 
                             path_col = 'path',
                            y_col = 'labels', 
                            target_size = IMG_SIZE,
                            batch_size = 64)

valid_gen = flow_from_dataframe(core_idg, valid_df, 
                             path_col = 'path',
                            y_col = 'labels', 
                            target_size = IMG_SIZE,
                            batch_size = 64) # we can use much larger batches for evaluation
# used a fixed dataset for evaluating the algorithm
test_X, test_Y = next(flow_from_dataframe(core_idg, 
                               valid_df, 
                             path_col = 'path',
                            y_col = 'labels', 
                            target_size = IMG_SIZE,
                            batch_size = 64)) # one big batch

In [ ]:
t_x, t_y = next(train_gen)

In [ ]:
t_x.shape[1:]

# Vgg16

In [ ]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

import keras
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Model,Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
%matplotlib inline

In [ ]:
pretrained_model_1 = VGG16(include_top=False, input_shape=t_x.shape[1:])

In [ ]:
from keras import optimizers
base_model = pretrained_model_1 # Topless

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(num_classes, activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

In [ ]:
model.fit_generator(train_gen,steps_per_epoch=100,validation_data = (test_X, test_Y), 
                                  epochs = 30)

# Test Data Preparation

In [ ]:
test_image_paths = glob('../input/test/*.jpg', recursive=True)

In [ ]:
test_image_paths[0:3]

In [ ]:
X_test = pd.DataFrame()

In [ ]:
X_test['path'] = test_image_paths

In [ ]:
X_test['labels'] = X_test['path'].map(lambda x: os.path.splitext(os.path.basename(x))[0])

In [ ]:
X_test[0:3]

In [ ]:
test_gen = flow_from_dataframe(core_idg, X_test, 
                             path_col = 'path',
                            y_col = 'labels', 
                            target_size = IMG_SIZE,
                            batch_size = 256) # we can use much larger batches for evaluation

In [ ]:
len(X_test)

In [ ]:
pred_Y =  model.predict_generator(test_gen,verbose = 1)

In [ ]:
len(test_image_paths)

In [ ]:
len(pred_Y)

In [ ]:
predictions = pred_Y[:,1]

In [ ]:
predictions[0:3]

In [ ]:
len(predictions)

In [ ]:
submission=pd.DataFrame()

In [ ]:
submission["id"]=X_test["labels"]

In [ ]:
submission["label"] = predictions

In [ ]:
submission.to_csv("predictions.csv",index=False)